In [285]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.neighbors import KNeighborsRegressor
from statsmodels.regression.linear_model import OLS
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam
import keras.backend as K
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score, make_scorer
from sklearn.model_selection import cross_val_score, LeaveOneOut
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import make_scorer
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
from sklearn.linear_model import *
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))


# Load the data
df_train = pd.read_csv('train_descriptors.csv')
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_train = df_train.drop(columns=["Compound No.","SMILES","logP","ring_count"])
# Extract the "MW" column
MW = df_train["MW"]

# Min-Max normalization
min_max_scaler = preprocessing.MinMaxScaler()
MW_normalized = min_max_scaler.fit_transform(MW.values.reshape(-1,1))

# Add the normalized column back to the dataframe
df_train["MW"] = MW_normalized
# Extract the "MW" column
TPSA = df_train["TPSA"]

# Min-Max normalization
min_max_scaler = preprocessing.MinMaxScaler()
TPSA_normalized = min_max_scaler.fit_transform(TPSA.values.reshape(-1,1))

# Add the normalized column back to the dataframe
df_train["TPSA"] = TPSA_normalized
# Split the training data into features and labels
X_train = df_train.drop(columns=['pIC50 (IC50 in microM)'])
y_train = df_train['pIC50 (IC50 in microM)']

X_train.to_csv('submission_to_submit3.csv', index=False)
def preprocess_data(X):
  imputer = SimpleImputer(strategy='mean')
  X = imputer.fit_transform(X)
  return X
# Preprocess the data
X_train = preprocess_data(X_train)
X_train1=X_train
y_train1=y_train
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,test_size=0.2, random_state=0)
from statsmodels.stats.stattools import durbin_watson

# Fit a linear regression model to the data
model = OLS(y_train, X_train).fit()

# Perform the Durbin-Watson test
dw_stat = durbin_watson(model.resid)

# Print the Durbin-Watson statistic
print("Durbin-Watson statistic: ", dw_stat)

# Check for autocorrelation
if dw_stat < 2 or dw_stat > 2:
    print("There is autocorrelation in the data, the model is likely non-linear")
else:
    print("There is no autocorrelation in the data, the model is likely linear")
    
# Define the model
model =RandomForestRegressor(n_estimators=100, random_state=0)
df_train.to_csv('submission_to_submit7.csv', index=False)
# Fit the model to the training data
model.fit(X_train, y_train)
df_train.to_csv('submission_to_submit8.csv', index=False)


# Make predictions on the validation set
predictions = model.predict(X_val)

# Calculate the mean squared error
mse = mean_squared_error(y_val, predictions)

# Take the square root of the mean squared error to get the root mean squared error
rmse = np.sqrt(mse)
print("RMSE:", rmse)
# Calculate the R-squared score
r2 = r2_score(y_val, predictions)

# print the R2 score
print("R2 score: ", r2)
df_train.to_csv('submission_to_submit5.csv', index=False)

Durbin-Watson statistic:  2.001082570957702
There is autocorrelation in the data, the model is likely non-linear
RMSE: 0.4983541724976178
R2 score:  0.5273482148664972


In [283]:
from scipy.stats import pearsonr
corr = []
p_value = []

for i in range(X_train.shape[1]):
    corr_i, p_value_i = pearsonr(X_train[:,i], y_train)
    corr.append(corr_i)
    p_value.append(p_value_i)

print("Correlation coefficients: ", corr)
print("p-values: ", p_value)

Correlation coefficients:  [-0.18933836288699657, -0.34838970300314015, -0.19349113512366997, -0.29007117631367796, -0.20230714599546026, -0.4903296543807421]
p-values:  [0.1248937474905164, 0.003863057621510686, 0.11668194878397109, 0.0172613760599387, 0.10062782983280501, 2.531412351949268e-05]


In [287]:
df_test = pd.read_csv('test_descriptors.csv')
df_1=df_test['SMILES']
df_test = df_test.drop(columns=["SMILES","logP","ring_count"])
# Extract the "MW" column
MW = df_test["MW"]

# Min-Max normalization
min_max_scaler = preprocessing.MinMaxScaler()
MW_normalized = min_max_scaler.fit_transform(MW.values.reshape(-1,1))

# Add the normalized column back to the dataframe
df_test["MW"] = MW_normalized
# Extract the "MW" column
TPSA = df_test["TPSA"]

# Min-Max normalization
min_max_scaler = preprocessing.MinMaxScaler()
TPSA_normalized = min_max_scaler.fit_transform(TPSA.values.reshape(-1,1))

# Add the normalized column back to the dataframe
df_test["TPSA"] = TPSA_normalized
test_predictions = model.predict(df_test)
df_test['pIC50 (IC50 in microM)'] = test_predictions
df_test = df_test[['pIC50 (IC50 in microM)']]
# Merge the columns of the two dataframes
df_test = pd.concat([df_1, df_test], axis=1)
df_test.to_csv('missing_smiles.csv', index=False)

C:\Users\deepa\anaconda3\envs\Python35\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [249]:
def create_model():
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    return model

model = RandomForestRegressor(n_estimators=100, random_state=0)

# Create an instance of the KFold()
kfold = KFold(n_splits=5)

# create a scoring function
scorer = make_scorer(r2_score)

# calculate the LOO-Q2 score
loo_q2 = cross_val_score(model, X_train1, y_train1, cv=kfold, scoring=scorer)
# print the LOO-Q2 score
print("LOO-Q2 score: ", loo_q2.mean())


LOO-Q2 score:  0.16556470127089237
